The starting point of the image classification section was to choose an architecture that would work properly for this specific problem. The established approach was to consider the models available in the Keras library, and benchmark some of them to see which one performs the best. The main factor for the decision was the combination of size of the model and the number of layers. We avoided those with big size and a high number of layers because the training data that we have is quite reduced, just 749 initial images to work with. If we used those, the training would have been slower, we might have come across OOM issues and smaller networks tend to better describe smaller sets of data. That is why we opted for models with low-medium size + a decent number of layers like Xception, InceptionV3, MobileNet or MobileNetV2; and others like VGG16, which is really heavy but just has 16 layers. It would have been possible to try even more architectures, but since time was limited, we focused on those.

The choice of cross-entropy as the loss function is already explained in its correspondent section, but there was a moment in which we had to decide between binary cross-entropy or categorical cross-entropy. Even though this last one sounded better for the problem we were facing, it works way better when you have one-hot vectors as predictions. Since our prediction vectors can have more than one "**1**" value because it is a multi-label problem, it was decided to go for binary cross-entropy instead. Related to this, for the metrics used during training we developed our own *keras.Metric* since the accuracies provided by *keras.metrics* weren't working as we wanted.

Moving on to the learning parameters, we started with a standard value for the learning rate with $10^-^5$. From that, we did an oriented but general parameter search that led us to use $10^-^3$ for models with just transfer learning, $10^-^4$ + $10^-^7$ for those that also got fine-tuning performed, and $0.5$ for those in which the weights had to be learned from scratch. In a similar fashion as the models themselves, with more time we search would have been more exhaustive, but the overall results were already decent with the ones finally used. When it comes to batch size, we used batches of 32 whenever possible since larger batches would cause OOM issues on some architectures. This just changes for the *learning from scratch cases*, because they needed way more epochs to train until convergence and if maintained the same it would have been too slow.

Even with all these features, the main problem we faced was overfitting. Not just when training from scratch, which is logical, but also with the two other approaches, even though they had a way better performance for sure. That is why we thought of performing basic data augmentation to increase the number of training images and with that, try to get better results. We created an augmentation function that, using the tools TensorFlow provides in *tf.image*, would create two additional images from an original one mainly playing with random values of contrast, saturation and brightness. By doing so, we would have $2,247$ training images instead of just $749$. And just in case it didn't work, we also implemented another more drastic function where we would randomly generate between 1 and 4 additional images per original one with flipping techniques apart from the previously mentioned parameters.

Bad news is that neither of those functions improved the overall score of the Xception model that much. There was a slight improvement of $0.03$ for the adapted accuracy metric with the first augmentation function, but we didn't include it because it didn't have the positive impact on overfitting we were searching for. Nonetheless, both these functions are shown below, so that it can be seen how we implemented this approach. Another set of measures that could improve the performance of the model are the threshold tuning with the Dice score or the argmax "hack", but they did not have an impact large enough so that it would have been worth implementing them all. Also, the argmax "hack" has to be cautiously supervised, because it can lead to either a better score or more false positives. However, we value these ideas as positives that could lead altogether to an increase of around $0.07$. The Dice score is already quite high, so there is not much room for improvement left there.

The final results with an adapted accuracy of $0.56$ or so and a manually-coded Dice score of $0.94$ show what seems a solid performance overall. We wouldn't consider it to be ready for real-world deployment yet because the number of false negatives is still noticeable, but at the same time it is a good minimum viable product. If we had more time to continue working on it, having a deeper search about architectures *(especially those that are outside the Keras library)* would have a big impact on the final performance. If we sum that to a broader hyperparameter search and the additions we came up with, the model could get really accurate.

Function for the starting data augmentation:

In [ ]:
import random

def augment(X_train, y_train):
    original_train_images = X_train.copy()
    for i in tqdm(range(len(original_train_images))):
        input_img = original_train_images[i]
        # augment 1
        r_val = random.uniform(1.25, 2)
        aug1 = tf.image.adjust_saturation(input_img, r_val)
        aug1 = tf.image.adjust_contrast(aug1, r_val)
        # augment 2
        aug2 = tf.image.flip_left_right(input_img)
        aug2 = tf.image.adjust_brightness(aug2, 0.2)

        for new_img in [aug1, aug2]:
            new_img = np.expand_dims(new_img, axis=0)
            X_train = np.concatenate((X_train, new_img), axis=0)
            y_train = np.concatenate((y_train, np.expand_dims(y_train[i], axis=0)), axis=0)

    return X_train, y_train

Function for the *drastic* data augmentation:

In [ ]:
def drastic_augment(X_train, y_train):
    original_train_images = X_train.copy()
    for i in tqdm(range(len(original_train_images))):
        input_img = original_train_images[i]
        list_augs, r_val = [], random.uniform(1.25, 2)

        # augment 1
        aug1 = tf.image.adjust_saturation(input_img, r_val)
        list_augs.append(tf.image.adjust_contrast(aug1, r_val))
        if r_val > 1.43: # augment 2
            aug2 = tf.image.flip_left_right(input_img)
            list_augs.append(tf.image.adjust_brightness(aug2, 0.25))
        if r_val > 1.61: # augment 3
            list_augs.append(tf.image.rot90(input_img))
        if r_val > 1.8: # augment 4
            aug4 = tf.image.flip_up_down(input_img)
            aug4 = tf.image.adjust_brightness(aug4, 0.25)
            list_augs.append(tf.image.adjust_contrast(aug4, r_val))

        for new_img in list_augs:
            new_img = np.expand_dims(new_img, axis=0)
            X_train = np.concatenate((X_train, new_img), axis=0)
            y_train = np.concatenate((y_train, np.expand_dims(y_train[i], axis=0)), axis=0)

    return X_train, y_train